1: Setup + carga de datos

In [1]:
import os
import pandas as pd
import tiktoken
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
USE_MOCK = os.getenv("USE_MOCK", "true").lower() == "true"

# Cargar propuestas desde CSV
try:
    df = pd.read_csv("../data/propuestas_civicdao.csv")
    print("✅ Propuestas cargadas:")
    display(df)
except FileNotFoundError:
    print("❌ Error: El archivo '../data/propuestas_civicdao.csv' no existe.")
    print("👉 Por favor, crea el archivo en la carpeta 'data/' con el formato correcto.")
    df = None  # Para evitar errores posteriores

✅ Propuestas cargadas:


,id,titulo,descripcion
0,1,Redistribución de fondos comunitarios,Proponemos mover el 20% del fondo operativo a ...
1,2,Actualización del contrato de gobernanza,Se propone actualizar el smart contract de vot...
2,3,Taller de onboarding para nuevos miembros,Organizar un evento mensual para explicar cómo...
3,4,Integración con Snapshot para votaciones,Conectar nuestro foro con Snapshot para facili...


 *pruebas ( tiktoken)

In [2]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")
tokens = encoding.encode("Hola desde Windows!")
print(f"✅ Tokens: {len(tokens)}")

✅ Tokens: 4


2: Función para contar tokens

In [3]:
def count_tokens(text: str, model: str = "gpt-4o-mini") -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

3: Prompt SIMPLE 

In [4]:
# === DEFINIR FUNCIÓN PARA CONTAR TOKENS ===
import tiktoken

def count_tokens(text: str, model: str = "gpt-4o-mini") -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# === GENERAR PROMPT SIMPLE ===
def simple_prompt(proposal: str) -> str:
    return f"Clasifica esta propuesta DAO en una de estas categorías: GOV, FIN, TEC, COM. Solo responde con la categoría. Propuesta: {proposal}"

# Usar la primera propuesta como ejemplo
prop = df.iloc[0]["descripcion"]
prompt_simple = simple_prompt(prop)

print("Prompt simple:\n", prompt_simple)
print("Tokens:", count_tokens(prompt_simple))


Prompt simple:
 Clasifica esta propuesta DAO en una de estas categorías: GOV, FIN, TEC, COM. Solo responde con la categoría. Propuesta: Proponemos mover el 20% del fondo operativo a iniciativas de inclusión digital en barrios marginados. Presupuesto: 15k USDC. Duración: 6 meses.
Tokens: 65


4: Prompt ESTRUCTURADO (con formato JSON)

In [5]:
def structured_prompt(proposal: str) -> str:
    return f"""
Clasifica la siguiente propuesta de DAO.
Categorías: GOV (gobernanza), FIN (finanzas), TEC (tecnología), COM (comunidad).
Responde SOLO en formato JSON: {{"categoria": "XXX", "confianza": 0.XX}}

Propuesta: {proposal}
""".strip()

prompt_struct = structured_prompt(prop)
print("Prompt estructurado:\n", prompt_struct)
print("Tokens:", count_tokens(prompt_struct))

Prompt estructurado:
 Clasifica la siguiente propuesta de DAO.
Categorías: GOV (gobernanza), FIN (finanzas), TEC (tecnología), COM (comunidad).
Responde SOLO en formato JSON: {"categoria": "XXX", "confianza": 0.XX}

Propuesta: Proponemos mover el 20% del fondo operativo a iniciativas de inclusión digital en barrios marginados. Presupuesto: 15k USDC. Duración: 6 meses.
Tokens: 96


5: Prompt FAST (comprimido + batching)

In [6]:
# === DEFINIR RESUMENES ===
# Si estás en modo MOCK, usa este resumen simulado
mock_summary = """
Se propone usar 5,000 USDC del fondo comunitario para realizar 3 talleres de alfabetización digital en el barrio San José, en colaboración con la biblioteca local. Beneficia directamente a vecinos sin acceso a formación digital, especialmente adultos mayores y jóvenes. La iniciativa dura 2 meses y su éxito se medirá por la asistencia y satisfacción de los participantes. Riesgo principal: baja participación si no hay difusión adecuada en la comunidad. Requiere voto urgente porque implica el uso de fondos colectivos y debe ejecutarse antes del próximo trimestre.
""".strip()

# Ejemplo: summary = "Resumen generado por la API..."

# === GENERAR PROMPT PARA IMAGEN ===
def build_image_prompt(summary_text: str) -> str:
    return f"""
Infographic for civic decision-making. Show: "Impacto de la propuesta" vs. "Situación actual". 
Use simple flat design, civic colors (blue, green, white), clear icons (people, laptop, calendar, survey form). 
No photorealistic, no complex backgrounds. High contrast, minimal text. Style: accessible, suitable for community forum.

Resumen de la propuesta: {summary_text[:150]}...
""".strip()

# Usar el resumen correcto según el modo
summary_to_use = mock_summary if USE_MOCK else summary

image_prompt = build_image_prompt(summary_to_use)

print("📌 Prompt para imagen (cópialo en Bing Image Creator):\n")
print(image_prompt)


try:
    summary_to_use = summary  # Si está definido
except NameError:
    summary_to_use = mock_summary  # Fallback si no está definido

📌 Prompt para imagen (cópialo en Bing Image Creator):

Infographic for civic decision-making. Show: "Impacto de la propuesta" vs. "Situación actual". 
Use simple flat design, civic colors (blue, green, white), clear icons (people, laptop, calendar, survey form). 
No photorealistic, no complex backgrounds. High contrast, minimal text. Style: accessible, suitable for community forum.

Resumen de la propuesta: Se propone usar 5,000 USDC del fondo comunitario para realizar 3 talleres de alfabetización digital en el barrio San José, en colaboración con la bibl...


6: Generar resumen (texto-texto)

In [ ]:
def summary_prompt(proposal: str) -> str:
    return f"""
Resume en 1 línea para votación rápida: {proposal}
"""

summary_p = summary_prompt(prop)
print("Prompt para resumen:\n", summary_p)

 7: Simulación de resultados (modo MOCK)

In [7]:
if USE_MOCK:
    print("\n--- RESULTADOS (MOCK) ---")
    print("Clasificación simple: FIN")
    print("Clasificación estructurada: {{'categoria': 'FIN', 'confianza': 0.95}}")
    print("Clasificación FAST (batch): FIN\nTEC")
    print("Resumen: Redistribución de 15k USDC a iniciativas de inclusión digital durante 6 meses.")
    
    # Prompt para imagen (texto-imagen)
    print("\n📌 Prompt para imagen (Bing Image Creator):")
    print('Infographic: "Propuesta FIN: 15k USDC → Inclusión Digital". Icons: money bag, people, laptop. Flat design, civic colors, high contrast.')


--- RESULTADOS (MOCK) ---
Clasificación simple: FIN
Clasificación estructurada: {{'categoria': 'FIN', 'confianza': 0.95}}
Clasificación FAST (batch): FIN
TEC
Resumen: Redistribución de 15k USDC a iniciativas de inclusión digital durante 6 meses.

📌 Prompt para imagen (Bing Image Creator):
Infographic: "Propuesta FIN: 15k USDC → Inclusión Digital". Icons: money bag, people, laptop. Flat design, civic colors, high contrast.


 8: Análisis de costos (markdown)

## 💰 Análisis de costos estimados (por 100 propuestas)

| Técnica                 | Llamadas | Tokens promedio | Tokens totales | Costo estimado (GPT-4o-mini) |
|-------------------------|----------|------------------|----------------|-------------------------------|
| Prompt simple           | 100      | 80               | 8,000          | ~$0.012                       |
| Prompt estructurado     | 100      | 150              | 15,000         | ~$0.0225                      |
| **Prompt FAST (batch 10)** | **10**   | **600**          | **6,000**      | **~$0.009** ✅                |

> **Conclusión**: El batching con prompts comprimidos reduce costos y llamadas, ideal para CivicDAO a escala.